In [1]:
import datetime
import glob
import HWT_mode_train
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pdb
import pickle
import re
import scalar2vector
import show_mode_examples
import sys
from tensorflow.keras.models import load_model
import webbrowser

In [2]:
min_confidence= 1
debug         = True
field         = "REFL_COM"
idir          = "/glade/scratch/ahijevyc"
segmentations = ["hyst"]
nmem          = 100
nns = [f"/glade/work/ahijevyc/NSC_objects/nn/nn_basic.Q1.S1.D1.alllabel.confmin1.ep35.bs10_{i}" for i in range(nmem)]

In [3]:
dfs = pd.DataFrame()
for segmentation in segmentations:
    searchstr = idir+"/track_data_ncarstorm_3km_"+field+"_"+segmentation+"_csv/track_step_NCARSTORM_d01_2*.csv"
    cfiles = glob.glob(searchstr)
    nfiles = len(cfiles)
    if nfiles == 0:
        print("no files match "+searchstr)
        sys.exit(1)

    print("reading",nfiles,"csv files matching "+searchstr) 
    df = [pd.read_csv(cfile, parse_dates=["Run_Date","Valid_Date"]).set_index("Step_ID") for cfile in cfiles]
    df = pd.concat(df)
    df['segmentation'] = segmentation
    dfs = dfs.append(df)

reading 539 csv files matching /glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/track_step_NCARSTORM_d01_2*.csv


In [10]:
dfs

,Track_ID,Ensemble_Name,Ensemble_Member,Run_Date,Valid_Date,Forecast_Hour,Valid_Hour_UTC,Duration,Centroid_Lon,Centroid_Lat,...,SHR1-potential_max,SHR6-potential_min,SHR6-potential_mean,SHR6-potential_max,10_min,10_mean,10_max,10-potential_min,10-potential_mean,10-potential_max
Step_ID,,,,,,,,,,,,,,,,,,,,,
d01_REFL_COM_20120601-0000_12_14_000_02,d01_REFL_COM_20120601-0000_12_14_000,NCARSTORM,d01,2012-06-01,2012-06-01 14:00:00,14,14,3,-82.46095,26.42896,...,28.349318,24.981147,18.586976,49.999282,21.850847,3.086543,18.654094,22.163152,2.968645,25.896632
d01_REFL_COM_20120601-0000_12_14_009_02,d01_REFL_COM_20120601-0000_12_14_009,NCARSTORM,d01,2012-06-01,2012-06-01 14:00:00,14,14,3,-100.93421,35.35696,...,15.699488,22.046396,20.110449,20.064162,7.487535,4.076999,8.503275,4.627682,3.975685,8.729200
d01_REFL_COM_20120601-0000_12_15_011_02,d01_REFL_COM_20120601-0000_12_15_011,NCARSTORM,d01,2012-06-01,2012-06-01 14:00:00,14,14,3,-82.98925,40.12875,...,15.240262,5.870232,19.971968,31.386378,5.068914,4.989826,13.893835,5.421277,5.899144,8.266592
d01_REFL_COM_20120601-0000_12_15_011_03,d01_REFL_COM_20120601-0000_12_15_011,NCARSTORM,d01,2012-06-01,2012-06-01 15:00:00,15,15,4,-82.53562,40.61086,...,23.330629,8.539582,19.625907,35.786161,11.244319,3.156178,14.372744,10.161929,5.883077,12.156734
d01_REFL_COM_20120601-0000_13_14_017_01,d01_REFL_COM_20120601-0000_13_14_017,NCARSTORM,d01,2012-06-01,2012-06-01 14:00:00,14,14,2,-85.03456,30.91215,...,24.755325,10.238703,10.955127,25.246987,6.409958,3.086154,16.302912,6.721707,5.044744,16.447132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d01_REFL_COM_20120329-0000_32_32_100_00,d01_REFL_COM_20120329-0000_32_32_100,NCARSTORM,d01,2012-03-29,2012-03-30 08:00:00,32,8,1,-91.12417,41.11106,...,26.123400,15.302371,20.231328,30.929812,5.256132,9.183833,14.376890,8.878330,9.938424,12.568805
d01_REFL_COM_20120329-0000_32_32_101_00,d01_REFL_COM_20120329-0000_32_32_101,NCARSTORM,d01,2012-03-29,2012-03-30 08:00:00,32,8,1,-90.36763,42.86855,...,36.081972,4.419816,19.017905,36.080609,13.485997,4.212757,12.488940,11.542974,5.631038,9.905972
d01_REFL_COM_20120329-0000_33_34_106_01,d01_REFL_COM_20120329-0000_33_34_106,NCARSTORM,d01,2012-03-29,2012-03-30 10:00:00,34,10,2,-88.11459,43.38630,...,36.309148,7.420613,25.467509,43.642681,17.649855,7.031526,14.856009,15.283757,7.099781,13.730119


In [5]:
HWT_mode_train.label_longname_dict()


{'Q1': 'QLCS\nQ1+Q2+S2', 'S1': 'Supercell\nS1+S3', 'D1': 'Disorganized\nD1+D2'}

In [6]:
features = HWT_mode_train.get_features(suite="basic")

# Derive vector magnitude, drop scalar components - already done earlier?
for f in features:
    if f not in dfs.columns:
        print(f, "not in dataframe. run scalar2vector.uvmagnitude?")

dfs = scalar2vector.uvmagnitude(dfs, debug=debug)

# Only keep objects with significant negative or positive UH. Those were the only ones labeled.
dfs=dfs[(dfs.UP_HELI_MAX_max >= 25) | (dfs.UP_HELI_MIN_min <= -25)]

dfs=dfs[(dfs.Forecast_Hour >= 14) & (dfs.Forecast_Hour <=34)]
#dfs = dfs[dfs.Run_Date.dt.year == 2013]

scaler_file = 'scaler.pkl'
if os.path.exists(scaler_file):
    scaler = pickle.load(open(scaler_file,'rb'))
else:
    scaler = dfs.describe()
    pickle.dump(scaler, open(scaler_file,'wb'))
    sys.exit(0)

scaled_dfs = (dfs.select_dtypes(include="number") - scaler.loc["mean"])/scaler.loc["std"]



10_max not in dataframe. run scalar2vector.uvmagnitude?
10-potential_max not in dataframe. run scalar2vector.uvmagnitude?
SHR6-potential_max not in dataframe. run scalar2vector.uvmagnitude?
SHR6-potential_mean not in dataframe. run scalar2vector.uvmagnitude?


In [7]:
features

['UP_HELI_MAX_max',
 'major_axis_length',
 'UP_HELI_MAX03_max',
 'area',
 'minor_axis_length',
 'eccentricity',
 'UP_HELI_MAX_mean',
 'UP_HELI_MAX01_max',
 'UP_HELI_MAX01_mean',
 'UP_HELI_MAX03_mean',
 'REFD_MAX_max',
 'RVORT1_MAX_max',
 'RVORT5_MAX_max',
 'REFL_1KM_AGL_max',
 '10_max',
 '10-potential_max',
 'SHR6-potential_max',
 'SHR6-potential_mean',
 'WSPD10MAX_max',
 'UP_HELI_MIN_mean',
 'Centroid_Lat',
 'Centroid_Lon']

In [8]:
scaler

,Forecast_Hour,Valid_Hour_UTC,Duration,Centroid_Lon,Centroid_Lat,Centroid_X,Centroid_Y,Storm_Motion_U,Storm_Motion_V,UP_HELI_MAX_mean,...,SHR6-potential_min,SHR6-potential_mean,SHR6-potential_max,_BUNK-potential_min,_BUNK-potential_mean,_BUNK-potential_max,orientation_sin,orientation_cos,Local_Solar_Hour_sin,Local_Solar_Hour_cos
count,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000,1.542900e+04,1.542900e+04,15429.000000,15429.000000,15429.000000,...,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000,15429.000000
mean,23.620196,11.874522,1.881587,-90.598859,37.524268,9.144563e+05,1.282886e+04,13996.111219,-1909.974723,7.137842,...,14.589041,17.582686,27.950752,9.385342,8.895511,14.590284,0.211555,0.036539,0.025102,0.010069
std,5.264881,8.457120,1.328344,10.279641,5.832027,8.991455e+05,6.372360e+05,39938.450642,42169.602092,5.797465,...,7.755940,7.983364,11.767379,4.627586,5.087453,7.939141,0.684938,0.696298,0.712965,0.700724
min,12.000000,0.000000,1.000000,-123.589250,24.527280,-1.816322e+06,-1.250144e+06,-363000.000000,-432000.000000,0.000020,...,0.016419,0.111599,0.393038,0.045484,0.057477,0.082633,-1.000000,-1.000000,-1.000000,-1.000000
25%,20.000000,3.000000,1.000000,-98.908940,33.039470,1.783605e+05,-4.853419e+05,-6000.000000,-21000.000000,3.475100,...,9.012396,11.979153,19.506009,6.049067,5.163662,8.603803,-0.414736,-0.642414,-0.699391,-0.692328
50%,23.000000,12.000000,1.000000,-89.412020,37.592500,9.930633e+05,2.357205e+04,15000.000000,0.000000,5.625860,...,13.567962,16.617830,26.324193,8.915246,7.956868,13.576805,0.392872,0.060420,0.074559,0.026752
75%,27.000000,20.000000,2.000000,-82.080140,42.066250,1.684983e+06,5.173990e+05,36000.000000,21000.000000,9.026710,...,19.014923,22.115767,35.082909,12.187820,11.632501,19.259472,0.856005,0.724491,0.732882,0.696657
max,35.000000,23.000000,12.000000,-69.119550,49.689540,2.447193e+06,1.262956e+06,426000.000000,393000.000000,110.650990,...,58.117469,58.679263,89.954325,32.135691,32.224153,49.337651,1.000000,1.000000,1.000000,1.000000


In [9]:
vyyyymmddhh = dfs["Run_Date"].dt.strftime("%Y%m%d%H/").values
preds = []
for nn in nns:
    model = load_model(nn, custom_objects={"brier_score":HWT_mode_train.brier_score, 
                                       "brier_skill_score":HWT_mode_train.brier_skill_score})
    preds.append(model.predict(scaled_dfs[features]))
    print(".",end="")
pred = np.array(preds).mean(axis=0)

....................................................................................................

In [11]:
scaled_dfs[features].mean()

UP_HELI_MAX_max        0.048907
major_axis_length     -0.020768
UP_HELI_MAX03_max      0.068953
area                  -0.024657
minor_axis_length     -0.028189
eccentricity          -0.002672
UP_HELI_MAX_mean       0.039805
UP_HELI_MAX01_max      0.080437
UP_HELI_MAX01_mean     0.080295
UP_HELI_MAX03_mean     0.054691
REFD_MAX_max           0.027332
RVORT1_MAX_max         0.062526
RVORT5_MAX_max         0.036064
REFL_1KM_AGL_max       0.019642
10_max                 0.070385
10-potential_max       0.042319
SHR6-potential_max     0.076363
SHR6-potential_mean    0.080870
WSPD10MAX_max          0.080246
UP_HELI_MIN_mean      -0.047625
Centroid_Lat          -0.068439
Centroid_Lon          -0.059936
dtype: float64

In [64]:
labels = HWT_mode_train.label_longname_dict()
labels = labels.keys()

xmlcode = f"<html><title>convective storm modes</title><body>"
xmlcode +=f"<h2>high probability convective storm modes</h2>"

topn = 5
pd.options.display.max_colwidth=200
xx = pd.concat([dfs.reset_index(), pd.DataFrame(pred, columns=labels)], axis='columns')
idir = "https://www2.mmm.ucar.edu/projects/ncar_ensemble/classifymode/mode_img/"
idir = "/glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/"
xx["fn"] = vyyyymmddhh + xx["Step_ID"] + "_+0.png"
for l in labels:
    xmlcode += f"\n<h3>{l}</h3>"
    xmlcode +="<table><tr>"
    rows = idir + xx.drop(columns="Step_ID").sort_values(by=l, ascending=False).reset_index(drop=True).head(topn)["fn"].values
    for row in rows:
        path, ext = os.path.splitext(row)
        xmlcode += f'<td><figure style="float:left;margin:0px">'
        xmlcode +=f'<figurecaption>{os.path.basename(path)[13:]}</figcaption>\n'
        xmlcode +=f'<img style="width:250px;height:440px;object-fit:cover;" src="file://{row}"</img>'
        xmlcode +=f'</figure></td>\n'
    xmlcode +='<br clear="all">'
    xmlcode +="</tr></table>"

xmlcode += '</body></html>'

print(xmlcode)

tmpfile = os.getenv("TMPDIR",os.getenv("HOME"))+"/topn.html"
with open(tmpfile,'w') as f:
    f.write(xmlcode)

print(tmpfile)


<html><title>convective storm modes</title><body><h2>high probability convective storm modes</h2>
<h3>Q1</h3><table><tr><td><figure style="float:left;margin:0px"><figurecaption>20151116-0000_31_31_089_00_+0</figcaption>
<img style="width:250px;height:440px;object-fit:cover;" src="file:///glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2015111600/d01_REFL_COM_20151116-0000_31_31_089_00_+0.png"</img></figure></td>
<td><figure style="float:left;margin:0px"><figurecaption>20110404-0000_29_29_145_00_+0</figcaption>
<img style="width:250px;height:440px;object-fit:cover;" src="file:///glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2011040400/d01_REFL_COM_20110404-0000_29_29_145_00_+0.png"</img></figure></td>
<td><figure style="float:left;margin:0px"><figurecaption>20110404-0000_23_23_118_00_+0</figcaption>
<img style="width:250px;height:440px;object-fit:cover;" src="file:///glade/scratch/ahijevyc/track_data_ncarstorm_3km_REFL_COM_hyst_csv/2011040400/d01_RE

In [51]:
pwd?

Docstring:
Return the current working directory path.

Examples
--------
::

  In [9]: pwd
  Out[9]: '/home/tsuser/sprint/ipython'
File:      /glade/work/ahijevyc/20201220_daa_casper/lib/python3.7/site-packages/IPython/core/magics/osm.py
